# Mistral Quantization and Finetuning

In [ ]:
!pip install -q accelerate
!pip install -qi https://pypi.org/simple/ bitsandbytes

In [ ]:
import torch
from transformers import AutoTokenizer, BitsAndBytesConfig, AutoModelForCausalLM

In [ ]:
model_name = 'mistralai/Mistral-7B-Instruct-v0.2'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

## Quantization
Quantization is done reduce memory footprint and perform faster inference while still retaining acceptable model performance. For this quantization, we will use bitandbytes

In [ ]:
# Use 4 bit compute 
use_4bit = True

# compute dtype for 4-bit models
compute_dtype = "float16"

# quantization type
quantization_type = 'nf4'

# use double quantization
use_nested_quant = False

In [ ]:
bnb_compute_dtype = getattr(torch, compute_dtype)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=quantization_type,
    bnb_4bit_compute_dtype=bnb_compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
)

In [ ]:
chat_input = tokenizer.encode_plus("[INST] Was Vivek Ramaswamy running for president ? [/INST]", return_tensors="pt")['input_ids'].to('cuda')

generated_ids = model.generate(chat_input, 
                               max_new_tokens=1000, 
                               do_sample=True, 
                               pad_token_id=tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)

## Langchain And Prompt Engineering

In [ ]:
!pip install -q langchain

In [ ]:
from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

In [ ]:
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task='text-generation',
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000
)

In [ ]:
prompt_template = """
### [INST]
Instruction: You are an expert political analyst with vast knowledge of the United States electoral process. You answer questions with 
certainty and you do not hallucinate. When unsure, you politely reply that you do not have sufficient knowledge to answer the user question.
You will generate new content by analysing the context supplied with each user question. Using this knowledge, answer the following questions.
Here is the context to help:

{context}

[/INST]
"""

In [ ]:
llm_pipeline = HuggingFacePipeline(pipeline=text_generation_pipeline)

In [ ]:
prompt = PromptTemplate(
    input_variables=['context', 'question']
    template=prompt_template
)

In [ ]:
llm_chain = LLMChain(llm=llm_pipeline, prompt=prompt)

In [ ]:
context = """
Vivek Ramaswamy, the multi-millionaire biotech entrepreneur and self-described intellectual godfather of the anti-woke movement, announced on Tuesday that he is running for president.
“We are in the middle of a national identity crisis,” he declared in an online video launching his campaign, offering that the current political climate constituted a form of “psychological slavery.”
Speaking straight to the camera, with an American flag draped in the background and a flag pin on his lapel, Ramaswamy framed his campaign as a broad counteroffensive to what he called the “woke left” — describing it as a threat to open speech, the free exchanging of ideas and American exceptionalism itself.
Ramaswamy is the third high-profile candidate to declare for the presidency in 2024. Though he filed forms with the FEC declaring he would be running on the Republican side of the aisle, his announcement video made no mention of the party itself — an indication that he hopes to frame his candidacy as outside the conventional political framework.
He has already done barnstorming in early nominating states, including Iowa, where he was well received even as some of the state’s political bigwigs professed to not having familiarity with the planks on which he was running.
Ramaswamy made his fortune in biotech investing, but he is best known for his appearances on Fox News and for the New York Times bestselling book he has written.
While his chances of securing the nomination are certainly long, Ramaswamy’s entry into the contest was greeted with a traditional flare from opposition Democrats. Shortly after he appeared on Fox News to elaborate on his decision to run, the Democratic National Committee sent out a statement.
“As Vivek Ramaswamy uses Tucker Carlson’s show to announce his campaign for president, one thing is clear: The race for the MAGA base is getting messier and more crowded by the day,” it read. “Over the next few months, Republicans are guaranteed to take exceedingly extreme positions on everything from banning abortion to cutting Social Security and Medicare and we look forward to continuing to ensure every American knows just how extreme the MAGA agenda is.”
"""

In [ ]:
llm_chain.invoke({"context": context,
                 "question": "Was Vivek Ramaswamy running for president in the 2024 general election ?"})

## Langchain and Chroma DB

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings

In [ ]:
from chromadb import HttpClient
chroma_client = HttpClient()

In [ ]:
vector_store = Chroma(client=chroma_client, 
                      embedding_function=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2"), 
                      collection_name='us-election-gpt')

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
query = 'What is Donald Trump up to lately?'
query_embedding = tokenizer(query)

In [ ]:
vector_store.similarity_search(query, k=5)

## Retriever with Chat History

In [ ]:
from langchain.prompts import MessagesPlaceholder, ChatPromptTemplate
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain, create_history_aware_retriever

In [ ]:
## Chat History retrieval prompt
chat_history_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

In [ ]:
chat_history_context_prompt = ChatPromptTemplate.from_messages(
    [('system', chat_history_system_prompt),
     MessagesPlaceholder('chat_history'),
     ('human', '{input}'),
    ]
)

In [ ]:
history_aware_retriever = create_history_aware_retriever(llm_pipeline, retriever, chat_history_context_prompt)

In [ ]:
## New question/answer prompt
chat_prompt = ChatPromptTemplate.from_messages(
    [('system', prompt_template),
     MessagesPlaceholder('chat_history'),
     ('human', '{input}')
    ]
)

In [ ]:
question_answer_chain = create_stuff_documents_chain(llm_pipeline, chat_prompt)

In [ ]:
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

### Redis Chat History
integrate Redis to store chat history

In [ ]:
REDIS_URL = "redis://localhost:6379/0"

In [ ]:
from langchain_community.chat_message_histories import RedisChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
def get_message_history(session_id: str) -> RedisChatMessageHistory:
    return RedisChatMessageHistory(session_id, url=REDIS_URL)

In [ ]:
rag_chain_llm = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
rag_chain_llm.invoke(
    {"input": "Did Vivek Ramaswamy run for president in the 2024 general election ?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]